# APPLYING PCA IMPROPERLY

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, f1_score

In [2]:
# firstly we get the mnist data as dataframe 
mnist_df = pd.read_csv("mnist_784.csv", sep=",")
mnist_df.head()    # class column is our label

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784,class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9


In [3]:
# let's look at the data info
mnist_df.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Columns: 785 entries, pixel1 to class
dtypes: int64(785)
memory usage: 419.2 MB


In [4]:
mnist_df['class'].value_counts(sort=True)   # the frequency of the classes (labels: 0-9)

1    7877
7    7293
3    7141
2    6990
9    6958
0    6903
6    6876
8    6825
4    6824
5    6313
Name: class, dtype: int64

In [5]:
X = mnist_df.drop(labels='class', axis=1)      #obtain the features from the raw data as X
X.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
y =  mnist_df['class']     # obtain the label from the raw data as y

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
     X, y,shuffle= True, train_size=0.7, random_state=42)

# random state is necessary for the same results.
# We determine the train percentage is 70%. You can use 80%, 90% percentage whatever you want.

In [8]:
scale_minmax = MinMaxScaler()
X_train_scaled = scale_minmax.fit_transform(X_train)
X_test_scaled = scale_minmax.transform(X_test)   # do not fit test data while scaling, it causes data leakege

# we scale data before PCA, PCA requires scaling (minmaxscaler: 0-1 scaling)

In [9]:
X_train.shape

(49000, 784)

In [10]:
X_test.shape

(21000, 784)

In [11]:
pca = PCA(0.95)   # we decided that 95% variance is sufficient
X_train_scaled_pca = pca.fit_transform(X_train_scaled)
X_test_scaled_pca = pca.fit_transform(X_test_scaled)   # we made this mistake intentionally 

In [12]:
pca.n_components_

157

In [13]:
X_train_scaled_pca.shape


(49000, 154)

In [14]:
X_test_scaled_pca.shape

(21000, 157)

In [15]:
clf_svc = SVC(kernel="rbf")
clf_svc.fit(X_train_scaled_pca, y_train)

SVC()

In [16]:
clf_svc.score(X_test_scaled_pca,y_test)

ValueError: X has 157 features, but SVC is expecting 154 features as input.

In [17]:
# What is the matter? let's try to get train score.
clf_svc.score(X_train_scaled_pca,y_train)

0.993265306122449

In [18]:
X_train_scaled_pca.shape

(49000, 154)

In [19]:
X_test_scaled_pca.shape

(21000, 157)

# ValueError: X has 157 features, but SVC is expecting 154 features as input.

As we can see that X_train_scaled_pca has 154 dimensions and X_test_scaled_pca has 157 dimensions
The reason of this inconsistency is misapplying pca (We fit the test data intentionally while applying pca.)
The model (with train data) learns that it has finally 154 dimensions although the test data has 157 dimensions after PCA with 0.95 variance. We can call it dimension inconsistency :)

# Let's correct our mistake

In [20]:
pca = PCA(0.95)   # we decide that 95% variance is sufficient
X_train_scaled_pca = pca.fit_transform(X_train_scaled)
X_test_scaled_pca = pca.transform(X_test_scaled)   # we do not use fit to avoid data leakege

In [21]:
X_train_scaled_pca.shape

(49000, 154)

In [22]:
X_test_scaled_pca.shape

(21000, 154)

#### Now we have the same dimensions 154=154

In [23]:
clf_svc_true = SVC(kernel="rbf")
clf_svc_true.fit(X_train_scaled_pca, y_train)
clf_svc_true.score(X_test_scaled_pca,y_test)

0.9800952380952381

In [24]:
f1_score(y_test, clf_svc_true.predict(X_test_scaled_pca), average='weighted')

0.9800801054906148